# 模型构造
自定义神经网络

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

In [3]:
X = torch.rand(2, 20)

In [11]:
# ============= 定义块 =============
class MLP(nn.Module): # 任何一个层or神经网络都是nn.Module的子类
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
        
    def forward(self, X):
        return self.out(F.relu(self.hidden(X))) 

# F.relu() 这里用nn.ReLU()会错误？ 
# nn.ReLU()是一个类，必须先实例化
'''
class MLP(nn.Module): # 任何一个层or神经网络都是nn.Module的子类
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
        self.relu = nn.ReLU() 
        
    def forward(self, X):
        return self.out(self.relu(self.hidden(X)))
'''

In [12]:
net = MLP() # 实例化类
net(X) # 自动调用forward.因为在类（class）中使用了__call__函数

tensor([[ 0.0019, -0.0919,  0.0149,  0.0375, -0.1196,  0.1278,  0.2666,  0.0913,
          0.0158,  0.1082],
        [ 0.2009,  0.0346,  0.1315, -0.0187,  0.1072,  0.0934,  0.2909,  0.1354,
          0.0894,  0.1067]], grad_fn=<AddmmBackward>)

In [13]:
# ============= nn.Sequential的内部实现 =============
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block # 按序的字典
    
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0596, -0.0940, -0.0724,  0.1808, -0.2780,  0.0514,  0.1904,  0.3403,
          0.0787,  0.0524],
        [ 0.1936,  0.0391, -0.1479,  0.2043, -0.2021,  0.1002,  0.2640,  0.2823,
         -0.0109,  0.1666]], grad_fn=<AddmmBackward>)

In [14]:
# ============= 在正向传播函数中执行代码 =============
# 这种形式下可以自定义很多内容
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=True)
        self.linear = nn.Linear(20, 20)
        
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.0684, grad_fn=<SumBackward0>)

In [16]:
# ============= 混合搭配各种组合块的方法 =============
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
        
    def forward(self, X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.2851, grad_fn=<SumBackward0>)

# 参数管理
## 定义好类后 参数如何访问?

In [17]:
# ============= 单隐藏层的多层感知机 =============
import torch 
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.0257],
        [ 0.0365]], grad_fn=<AddmmBackward>)

In [18]:
print(net[2].state_dict()) # 可以把nn.Sequential看作list

OrderedDict([('weight', tensor([[ 0.0550, -0.1881,  0.2363,  0.0447, -0.1045,  0.2708,  0.1435,  0.1868]])), ('bias', tensor([-0.2026]))])


In [20]:
print(type(net[2].bias))
print(net[2].bias) # 包含数据本身&梯度
print(net[2].bias.data, net[2].bias.grad)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.2026], requires_grad=True)
tensor([-0.2026]) None


In [21]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))


In [22]:
print(*[(name, param.shape) for name, param in net.named_parameters()])

('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [24]:
net.state_dict()['2.bias'].data

tensor([-0.2026])

In [26]:
# ============= 从嵌套块收集参数 =============
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4),
                        nn.ReLU())
def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.2218],
        [-0.2218]], grad_fn=<AddmmBackward>)

In [27]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [29]:
# ============= 初始化 =============
# 内置初始化
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01) # 表示替换而非返回
        nn.init.zeros_(m.bias)
        
net.apply(init_normal) # net中所有的Module都遍历一遍
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0013,  0.0091,  0.0079, -0.0058]), tensor(0.))

In [30]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [31]:
# 对某些块应用不同的初始化方法
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
        
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.0092,  0.2663,  0.1231, -0.4041])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [32]:
net[0].weight.data

tensor([[ 0.0092,  0.2663,  0.1231, -0.4041],
        [-0.4085, -0.0224,  0.2815, -0.5987],
        [ 0.5450, -0.4741,  0.4667,  0.0615],
        [-0.2436,  0.2077, -0.0925,  0.1365],
        [ 0.0660,  0.4972,  0.3616,  0.0497],
        [ 0.2788, -0.1422,  0.5875,  0.4511],
        [-0.3069, -0.1848,  0.5713,  0.6781],
        [-0.4754, -0.1582, -0.4420, -0.4890]])

## 参数绑定（共享参数）

In [33]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(),
                   shared, nn.ReLU(), nn.Linear(8, 1))
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0]) # 指向同一个实例

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
